# Config

In [53]:
import subprocess
import geopandas as gpd
import os
from google.cloud import storage
import glob
import shutil
import pathlib

# Transform To Earth Coordinates

In [49]:
os.environ["PROJ_IGNORE_CELESTIAL_BODY"] = "YES"

In [50]:
labels_mars = gpd.read_file("gs://esg-satelite-data-warehouse/mars/mars_labels/Orlov_TempeTerra_fault_catalogue.shp")

In [51]:
labels_earth = labels_mars.to_crs("epsg:4326")

In [54]:
local_folder = pathlib.Path("out")
local_folder.mkdir(exist_ok=False)

In [55]:
labels_earth.to_file("out/faultlines.shp")

In [56]:
for file in glob.glob("out/*"):
    extension = file.split(".")[-1]
    client = storage.Client()
    bucket = client.get_bucket("esg-satelite-data-warehouse")
    blob = bucket.blob("mars/mars_labels_processed/faultlines." + extension)
    blob.upload_from_filename(f"out/faultlines.{extension}")

In [57]:
shutil.rmtree("out")

# Transfer from GCP to EE

In [22]:
ee_table_path = f"projects/esg-satelite/assets/mars/labels/faultlines"
shapefile_post_path = "gs://esg-satelite-data-warehouse/mars/mars_labels_processed/faultlines"

In [23]:
result = subprocess.run(
    [
        "earthengine",
        "upload",
        "table",
        f"--asset_id={ee_table_path}",
        f"{shapefile_post_path}.shp"
    ],
    capture_output=True,
    check=True,
)
print(result.stdout.decode())
print(result.stderr.decode())

Started upload task with ID: EMSK2PUHZ4VPSMMHT6GZI7YK


